In [1]:
import pandas as pd
import numpy as np
import matplotlib
from tqdm import tqdm 

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion, make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import catboost as catb


from sklearn.metrics import precision_recall_curve, roc_curve, auc, f1_score, make_scorer, accuracy_score

import dask.dataframe as dd

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Этап 1. Подготовка train и test датасетов  

In [3]:
# уменьшение занимаемого объема памяти
def reduce_mem_usage(train_data):
    """ Изменение типов данных датасета для уменьшения занимаемого объема памяти.        
    """
    start_mem = train_data.memory_usage().sum() / 1024**2
    print('Использование памяти до обработки: {:.2f} MB'.format(start_mem))
    
    for col in train_data.columns:
        col_type = train_data[col].dtype
        
        if col_type != object:
            c_min = train_data[col].min()
            c_max = train_data[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    train_data[col] = train_data[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    train_data[col] = train_data[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    train_data[col] = train_data[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    train_data[col] = train_data[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    train_data[col] = train_data[col].astype(np.float32)
                else:
                    train_data[col] = train_data[col].astype(np.float64)
        else:
            train_data[col] = train_data[col].astype('object')

    end_mem = train_data.memory_usage().sum() / 1024**2
    print('Использование памяти после обработки: {:.2f} MB'.format(end_mem))
    print('Уменьшение занимаемого объема на: {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return train_data

In [4]:
train_data = pd.read_csv('data_train.csv', sep=",")
test_data = pd.read_csv('data_test.csv', sep=",")
df = dd.read_csv('features.csv', dtype='object', sep="\t")

In [5]:
train_data.shape, test_data.shape

((831653, 5), (71231, 4))

In [6]:
train_data.head(3)

,Unnamed: 0,id,vas_id,buy_time,target
0,0,540968,8.0,1537131600,0.0
1,1,1454121,4.0,1531688400,0.0
2,2,2458816,1.0,1534107600,0.0


In [7]:
test_data.head(3)

,Unnamed: 0,id,vas_id,buy_time
0,0,3130519,2.0,1548018000
1,1,2000860,4.0,1548018000
2,2,1099444,2.0,1546808400


In [8]:
df.head(3)

,Unnamed: 0,id,buy_time,0,1,2,3,4,5,6,...,243,244,245,246,247,248,249,250,251,252
0,0,2013026,1531688400,18.91002922530346,46.98088788147152,4.969214041441917,-1.3867982403657493,3.7917537065697497,-14.011789573383396,-16.0861800325671,...,-977.3738461013428,-613.7707921147527,-25.996268610410837,-37.63044838724547,-301.7477236706343,-25.83288857155014,-0.6944284888647783,-12.175933312768366,-0.4561398843397759,0.0
1,1,2014722,1539550800,36.690029225303476,152.40088788147153,448.0692140414418,563.8332017596342,463.84175370656976,568.9982104266168,-16.0861800325671,...,-891.3738461013428,-544.7707921147527,-20.99626861041084,48.369551612754535,80.2522763293657,-13.832888571550141,-0.6944284888647783,-1.1759333127683649,-0.4561398843397759,0.0
2,2,2015199,1545598800,-67.01997077469653,157.0508878814715,-63.18078595855808,178.10320175963426,-68.59824629343025,156.9982104266166,3.5138199674329016,...,-977.3738461013428,-613.7707921147527,-12.996268610410835,-37.63044838724547,10829.252276329364,-25.83288857155014,-0.6944284888647783,-12.175933312768366,-0.4561398843397759,0.0


In [9]:
train_data = train_data.drop(['Unnamed: 0'], axis=1)
test_data = test_data.drop(['Unnamed: 0'], axis=1)
df = df.drop(['Unnamed: 0'], axis=1)

In [10]:
train_data.dtypes

id            int64
vas_id      float64
buy_time      int64
target      float64
dtype: object

In [11]:
test_data.dtypes

id            int64
vas_id      float64
buy_time      int64
dtype: object

In [12]:
#Попробуем уменьшить используемый объем памяти для работы с train_data и test_data
print('Обрабатываем train_data')
train_data = reduce_mem_usage(train_data)
print()
print('Обрабатываем test_data')
test_data = reduce_mem_usage(test_data)

Обрабатываем train_data
Использование памяти до обработки: 25.38 MB
Использование памяти после обработки: 12.69 MB
Уменьшение занимаемого объема на: 50.0%

Обрабатываем test_data
Использование памяти до обработки: 1.63 MB
Использование памяти после обработки: 0.82 MB
Уменьшение занимаемого объема на: 50.0%


In [13]:
df.dtypes

id          object
buy_time    object
0           object
1           object
2           object
             ...  
248         object
249         object
250         object
251         object
252         object
Length: 255, dtype: object

In [14]:
df.dtypes

id          object
buy_time    object
0           object
1           object
2           object
             ...  
248         object
249         object
250         object
251         object
252         object
Length: 255, dtype: object

In [15]:
#проверим уникальность поля id
train_data['id'].is_unique, test_data['id'].is_unique

(False, False)

In [16]:
#выберем всех уникальных пользователей из train и test для того, чтобы выбрать из df только нужные данные
unique_users_train = set(train_data['id'].values)
unique_users_test = set(test_data['id'].values)
unique_users = unique_users_train | unique_users_test 
print(f'уникальных пользователей в train_data: {len(unique_users_train)}')
print(f'уникальных пользователей в test_data: {len(unique_users_test)}')
print(f'уникальных пользователей в двух датасетах: {len(unique_users)}')

уникальных пользователей в train_data: 806613
уникальных пользователей в test_data: 70152
уникальных пользователей в двух датасетах: 872577


In [17]:
#количество разделов, на которые поделен df
df.npartitions

352

In [18]:
df['id'] = df['id'].astype('int')

In [19]:
 df_update = pd.DataFrame()

# for part in tqdm(range(0, 3)):
for part in tqdm(range(0, df.npartitions)):
    df_to_pandas = df.get_partition(part).compute()
    df_filter = df_to_pandas['id'].isin(list(unique_users)) 
    df_part = df_to_pandas[df_filter] 
    df_update = pd.concat([df_update, df_part], ignore_index=True)

100%|████████████████████████████████████████████████████████████████████████████████| 352/352 [21:15<00:00,  3.62s/it]


In [20]:
columns_df_update_to_int = ['id', 'buy_time']
columns_df_update_to_float = df_update.columns.drop(columns_df_update_to_int).tolist()

In [21]:
df_update[columns_df_update_to_int] = df_update[columns_df_update_to_int].astype('int')
df_update[columns_df_update_to_float] = df_update[columns_df_update_to_float].astype('float')

In [22]:
#уменьшим занимаемый объем памяти у df_update
df_update = reduce_mem_usage(df_update)

Использование памяти до обработки: 1748.71 MB
Использование памяти после обработки: 877.80 MB
Уменьшение занимаемого объема на: 49.8%


In [23]:
df_update.dtypes

id            int32
buy_time      int32
0           float32
1           float32
2           float32
             ...   
248         float32
249         float32
250         float32
251         float32
252         float32
Length: 255, dtype: object

In [24]:
df_update.shape

(902389, 255)

In [25]:
df_update.head(2)

,id,buy_time,0,1,2,3,4,5,6,7,...,243,244,245,246,247,248,249,250,251,252
0,2046132,1534712400,300.820038,1599.480835,286.879211,1585.013184,281.461761,1563.908203,-16.08618,654.013916,...,-977.37384,-613.770813,-25.996269,-35.630447,-295.747711,-17.832888,-0.694429,-4.175933,-0.45614,0.0
1,2050810,1540760400,-86.209969,91.820885,-84.480789,110.333199,-89.898247,89.228210,-16.08618,-65.076096,...,-977.37384,-613.770813,-23.996269,190.369553,-286.747711,-25.832888,-0.694429,-12.175933,-0.45614,0.0


In [26]:
#сохраннеие предобработанного df_update
df_update.to_csv("df_update.csv", index=None)

In [27]:
train_data.head(2)

,id,vas_id,buy_time,target
0,540968,8.0,1537131600,0.0
1,1454121,4.0,1531688400,0.0


In [28]:
test_data.head(2)

,id,vas_id,buy_time
0,3130519,2.0,1548018000
1,2000860,4.0,1548018000


In [29]:
test_data['target'] = np.nan

In [30]:
data = pd.concat([train_data, test_data], ignore_index=True)

In [31]:
data.shape

(902884, 4)

In [32]:
data.dtypes

id            int32
vas_id      float32
buy_time      int32
target      float32
dtype: object

In [33]:
data = data.sort_values(by=['buy_time'])
df_update = df_update.sort_values(by=['buy_time'])

In [34]:
data_merge = pd.merge_asof(data, df_update, on="buy_time", by='id', direction="nearest")

In [35]:
data_merge.shape

(902884, 257)

In [36]:
#объединенный датасет 
data_merge

,id,vas_id,buy_time,target,0,1,2,3,4,5,...,243,244,245,246,247,248,249,250,251,252
0,1388074,2.0,1531083600,0.0,551.310059,708.170898,546.259216,695.363220,540.841736,674.258240,...,13.626154,-245.770798,-24.996269,-37.630447,-16.747725,-23.832888,-0.694429,-11.175933,-0.45614,0.0
1,338913,4.0,1531083600,0.0,-96.799973,100.290886,-100.570786,84.563202,-105.988243,63.458210,...,1787.626099,2063.229248,-25.996269,-17.630449,-256.747711,-25.832888,-0.694429,-12.175933,-0.45614,0.0
2,1552375,2.0,1531083600,0.0,-26.869970,172.370895,-28.100786,132.473206,-33.518246,111.368210,...,-977.373840,-613.770813,-25.996269,-37.630447,-271.747711,-22.832888,-0.694429,-9.175933,-0.45614,1.0
3,2516522,2.0,1531083600,1.0,-96.799973,-111.569115,-110.740784,-164.176804,-116.158249,-185.281784,...,-828.373840,-536.770813,-25.996269,-36.630447,-276.747711,-22.832888,-0.694429,-12.175933,-0.45614,1.0
4,3299896,1.0,1531083600,0.0,39.370029,-129.699112,25.429214,-178.136795,20.011753,-199.241791,...,-745.373840,-613.770813,-24.996269,200.369553,-21.747725,8.167111,-0.694429,-7.175933,-0.45614,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902879,3145392,6.0,1548018000,NaN,-94.259972,-92.079109,-108.200783,-144.686798,-113.618248,-165.791794,...,-977.373840,-613.770813,-25.996269,-37.630447,-306.747711,-25.832888,-0.694429,-12.175933,-0.45614,0.0
902880,2452188,2.0,1548018000,NaN,-90.689972,-174.939117,-104.630783,-227.546799,-110.048248,-248.651794,...,-918.373840,-613.770813,-25.996269,-37.630447,-305.747711,-25.832888,-0.694429,-12.175933,-0.45614,0.0
902881,3145390,2.0,1548018000,NaN,-91.719971,-79.779114,-105.660789,-123.906799,-111.078247,-145.011795,...,-977.373840,-613.770813,-25.996269,-37.630447,-306.747711,-25.832888,-0.694429,-12.175933,-0.45614,0.0
902882,1447357,6.0,1548018000,NaN,-76.729973,-351.689117,-69.020782,-334.966797,-74.438248,-356.071777,...,-977.373840,-613.770813,-25.996269,-37.630447,-306.747711,-25.832888,-0.694429,-12.175933,-0.45614,0.0


In [37]:
train_df = data_merge.loc[~data_merge['target'].isna()]
test_df = data_merge.loc[data_merge['target'].isna()]
test_df = test_df.drop('target', axis=1)

In [38]:
# для удобства поменяем порядок столбцов в датафрейме
train_df = train_df.reindex(columns=['id','vas_id','buy_time','0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41','42','43','44','45','46','47','48','49','50','51','52','53','54','55','56','57','58','59','60','61','62','63','64','65','66','67','68','69','70','71','72','73','74','75','76','77','78','79','80','81','82','83','84','85','86','87','88','89','90','91','92','93','94','95','96','97','98','99','100','101','102','103','104','105','106','107','108','109','110','111','112','113','114','115','116','117','118','119','120','121','122','123','124','125','126','127','128','129','130','131','132','133','134','135','136','137','138','139','140','141','142','143','144','145','146','147','148','149','150','151','152','153','154','155','156','157','158','159','160','161','162','163','164','165','166','167','168','169','170','171','172','173','174','175','176','177','178','179','180','181','182','183','184','185','186','187','188','189','190','191','192','193','194','195','196','197','198','199','200','201','202','203','204','205','206','207','208','209','210','211','212','213','214','215','216','217','218','219','220','221','222','223','224','225','226','227','228','229','230','231','232','233','234','235','236','237','238','239','240','241','242','243','244','245','246','247','248','249','250','251','252','target'])

In [39]:
# #сохраним два новых предобработанных датасета
train_df.to_csv("train_df.csv", index=None)
test_df.to_csv("test_df.csv", index=None)

Этап подготовки датасета для дальнейшего анализа данных закончен. Переходим к EDA и построениею Baseline.